Universidad del Valle de Guatemala

Security Data Science

Augusto Alonso

Douglas de León - 18037

# Laboratorio 1: Phishing Detection

# Get the necessary libraries

In [18]:
import sys
!{sys.executable} -m pip install pandas-profiling

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [129]:
import pandas as pd
import features_utils as fu
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [53]:
dataset = pd.read_csv("./dataset_pishing.csv")

# Pandas Profiling

In [30]:
profile = ProfileReport(dataset, title="Phishing dataset Profiling Report")
profile.to_file("initial_report.html")

Summarize dataset:   0%|          | 0/80 [00:00<?, ?it/s]

/Users/augustoalonso/Library/Python/3.9/lib/python/site-packages/missingno/missingno.py:250: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize)
/Users/augustoalonso/Library/Python/3.9/lib/python/site-packages/pandas_profiling/model/summary.py:194: UserWarning: There was an attempt to generate the bar missing values diagrams, but this failed.
    To hide this warning, disable the calculation
    (using `df.profile_report(missing_diagrams={"bar": False}`)
    If this is problematic for your use case, please report this as an issue:
    https://github.com/pandas-profiling/pandas-profiling/issues
    (include the error message: 'The number of FixedLocator locations (7), usually from a call to set_ticks, does not match the number of ticklabels (67).')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Observaciones

In [54]:
pd.set_option('max_columns', None)
dataset.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,4,4,3,3,3,11,11,6,5.750000,7.0,4.500000,0,0,0,0,0,0,17,0.529412,0.470588,0,0,0,0.875000,0,0.500000,0,0,80.000000,0,100.000000,0.000000,0,0,0,0.0,0,0,0,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,4,4,2,19,2,32,19,32,15.750000,19.0,14.666667,0,0,0,0,0,0,30,0.966667,0.033333,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,80.000000,20.000000,0,0,0,100.0,0,0,0,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,0,3,1,0,0,0,1,0,12,2,2,3,2,17,13,17,8.250000,8.4,8.142857,0,0,0,0,0,0,4,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,0.000000,0.000000,0,0,0,100.0,0,0,0,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,5,5,0,5,5,0,5.000000,5.0,0.000000,0,0,0,0,0,0,149,0.973154,0.026846,0,0,0,0.250000,0,0.250000,0,0,100.000000,0,96.428571,3.571429,0,0,0,62.5,0,0,0,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,6,3,3,3,4,11,7,11,6.333333,5.0,7.000000,0,0,0,0,0,0,102,0.470588,0.529412,0,0,0,0.537037,0,0.018519,1,0,76.470588,0,0.000000,100.000000,0,0,0,0.0,0,0,0,0,1,0,224,8175,8725,0,0,6,legitimate


In [55]:
dataset.columns.values

array(['url', 'ip', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path',
       'punycode', 'port', 'tld_in_path', 'tld_in_subdomain',
       'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix',
       'random_domain', 'shortening_service', 'path_extension',
       'nb_redirection', 'nb_external_redirection', 'length_words_raw',
       'char_repeat', 'shortest_words_raw', 'shortest_word_host',
       'shortest_word_path', 'longest_words_raw', 'longest_word_host',
       'longest_word_path', 'avg_words_raw', 'avg_word_host',
       'avg_word_path', 'phish_hints', 'domain_in_brand',
       'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
       'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks',
       'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
       'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors',
       'ratio_extErrors', 'login_form', 'external_favicon',
       'links_in_tags', 'submit_email', 'ratio_intMedia',
       'ratio_extMedi

In [56]:
pd.reset_option('max_columns')

Número de observaciones en la columna status

In [57]:
dataset["status"].value_counts()

legitimate    5715
phishing      5715
Name: status, dtype: int64

El dataset se encuentra balanceado en la columna objetivo (status).

## Feature generation

In [58]:
dataset['f1'] = dataset['url'].map(fu.f1) # Full URL length.
dataset['f2'] = dataset['url'].map(fu.f2) # Hostname length.
dataset['f4'] = dataset['url'].map(fu.f4) # Number of occurences of character '.' in URL.
dataset['f5'] = dataset['url'].map(fu.f5) # Number of occurences of character '-' in URL.
dataset['f6'] = dataset['url'].map(fu.f6) # Number of occurences of character '@' in URL.
dataset['f7'] = dataset['url'].map(fu.f7) # Number of occurences of character '?' in URL.
dataset['f8'] = dataset['url'].map(fu.f8) # Number of occurences of character '&' in URL.
dataset['f9'] = dataset['url'].map(fu.f9) # Number of occurences of character '|' in URL.
dataset['f10'] = dataset['url'].map(fu.f10) # Number of occurences of character '=' in URL.
dataset['f11'] = dataset['url'].map(fu.f11) # Number of occurences of character '_' in URL.
dataset['f12'] = dataset['url'].map(fu.f12) # Number of occurences of character '~' in URL.
dataset['f13'] = dataset['url'].map(fu.f13) # Number of occurences of character '%' in URL.
dataset['f14'] = dataset['url'].map(fu.f14) # Number of occurences of character '/' in URL.
dataset['f15'] = dataset['url'].map(fu.f15) # Number of occurences of character '*' in URL.
dataset['f16'] = dataset['url'].map(fu.f16) # Number of occurences of character ':' in URL.
dataset['f17'] = dataset['url'].map(fu.f17) # Number of occurences of character ',' in URL.
dataset['f18'] = dataset['url'].map(fu.f18) # Number of occurences of character ';' in URL.
dataset['f19'] = dataset['url'].map(fu.f19) # Number of occurences of character '$' in URL.
dataset['f20'] = dataset['url'].map(fu.f20) # Number of occurences of character '%20' or space in URL.
dataset['f25'] = dataset['url'].map(fu.f25) # Checks for HTTPS token in URL.
dataset['f26'] = dataset['url'].map(fu.f26) # Ratio of digits in full URL.
dataset['f27'] = dataset['url'].map(fu.f27) # Ratio of digits in hostname.

In [59]:
dataset.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,False,0.000000,0.0
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,False,0.220779,0.0
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,5,0,1,0,0,0,0,True,0.150794,0.0
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,False,0.000000,0.0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,False,0.000000,0.0


## Preprocesamiento
Realice las modificaciones necesarias para convertir la variable categórica status a una variable
binaria. Elimine la columna del dominio.

In [60]:
safe_copy = dataset.copy()

In [61]:
df_binary_status = pd.get_dummies(dataset["status"])
df_binary_status = df_binary_status.drop(["legitimate"], axis=1)
dataset = dataset.drop(["status"], axis=1)
dataset = pd.concat((dataset, df_binary_status), axis=1)
dataset = dataset.rename(columns={"phishing": "status"})
# Drop domain
dataset["status"]

0        0
1        1
2        1
3        0
4        0
        ..
11425    0
11426    1
11427    0
11428    0
11429    1
Name: status, Length: 11430, dtype: uint8

In [ ]:
dataset = dataset.drop(["url"], axis=1)

Ahora status es un binario donde 1 representa que tiene phishing y 0 que es legitimo.

# Visualización de resultados

Genere un reporte de perfil con la librería pandas_profiling. Analice el reporte y determine las
columnas que son constantes, o que no tienen una varianza alta con la columna status. Almacene su
reporte como una página html.

In [62]:
profile = ProfileReport(dataset, title="Phishing dataset after binary modification and features")
profile.to_file("status_binary_report.html")

Summarize dataset:   0%|          | 0/101 [00:00<?, ?it/s]

/Users/augustoalonso/Library/Python/3.9/lib/python/site-packages/missingno/missingno.py:250: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize)
/Users/augustoalonso/Library/Python/3.9/lib/python/site-packages/pandas_profiling/model/summary.py:194: UserWarning: There was an attempt to generate the bar missing values diagrams, but this failed.
    To hide this warning, disable the calculation
    (using `df.profile_report(missing_diagrams={"bar": False}`)
    If this is problematic for your use case, please report this as an issue:
    https://github.com/pandas-profiling/pandas-profiling/issues
    (include the error message: 'The number of FixedLocator locations (7), usually from a call to set_ticks, does not match the number of ticklabels (88).')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [74]:
print("Columnas con correlación abs > 0.25")
correlation_columns = []
no_correlation_columns = []
for column in filter(lambda c: c != "status", dataset.columns):
    corr = dataset['status'].corr(dataset[column])
    if abs(abs(corr) > 0.25):
        print("Correlación entre status y ", column)
        print(corr)
        correlation_columns.append(column)
    else:
        no_correlation_columns.append(column)

Columnas con correlación abs > 0.25
Correlación entre status y  ip
0.32169778238741015
Correlación entre status y  nb_www
-0.44346768751286336
Correlación entre status y  phish_hints
0.3353927377156515
Correlación entre status y  nb_hyperlinks
-0.3426282900378529
Correlación entre status y  domain_in_title
0.34280697299383556
Correlación entre status y  domain_age
-0.33188908389604205
Correlación entre status y  google_index
0.731170838126266
Correlación entre status y  page_rank
-0.5111371436645006
Correlación entre status y  f7
0.29431907084572934
Correlación entre status y  f26
0.3564210461092021


In [75]:
print("Variables con poca correlación")
for c in no_correlation_columns:
    print(c)

Variables con poca correlación
nb_com
nb_dslash
http_in_path
punycode
port
tld_in_path
tld_in_subdomain
abnormal_subdomain
nb_subdomains
prefix_suffix
random_domain
shortening_service
path_extension
nb_redirection
nb_external_redirection
length_words_raw
char_repeat
shortest_words_raw
shortest_word_host
shortest_word_path
longest_words_raw
longest_word_host
longest_word_path
avg_words_raw
avg_word_host
avg_word_path
domain_in_brand
brand_in_subdomain
brand_in_path
suspecious_tld
statistical_report
ratio_intHyperlinks
ratio_extHyperlinks
ratio_nullHyperlinks
nb_extCSS
ratio_intRedirection
ratio_extRedirection
ratio_intErrors
ratio_extErrors
login_form
external_favicon
links_in_tags
submit_email
ratio_intMedia
ratio_extMedia
sfh
iframe
popup_window
safe_anchor
onmouseover
right_clic
empty_title
domain_with_copyright
whois_registered_domain
domain_registration_length
web_traffic
dns_record
f1
f2
f4
f5
f6
f8
f9
f10
f11
f12
f13
f14
f15
f16
f17
f18
f19
f20
f25
f27


Para facilitar el analisis realizaremos un nuevo pandas profiles únicamente con las variables que tienen correlación esto con el motivo de poder examinar mas fácilmente dentro de estas cuales podrían tener correlación entre ellas y evitar sesgo. También poder revisar cuales son constantes.

In [76]:
# Create the dataset to feed the final profiles only with the correlation columns
final_dataset = dataset.drop(no_correlation_columns, axis=1)

In [77]:
final_profile = ProfileReport(final_dataset, title="Final Phishing profiler (only correlation columns)")
final_profile.to_file("final_report.html")

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Resultados profiling
Debido a que cargar el pandas profiling es relativamente pesado para el navegador causando que crashee y las correlaciones salen en un reporte muy pequeño de leer creamos la correlación de status con todas las columnas a mano

#### Variables que tienen correlación con status
Consideramos como variables con correlación únicamente las variables que su valor absoluto superara 0.25

<b>Variables con correlación positiva</b>

- ip : 0.32169778238741015
- phish_hints : 0.3353927377156515
- domain_in_title : 0.34280697299383556
- google_index : 0.731170838126266
- f7 : 0.29431907084572934
- f26 : 0.3564210461092021


<b>Variables con correlación negativa</b>

- nb_www: -0.44346768751286336
- nb_hyperlinks: -0.3426282900378529
- domain_age: -0.33188908389604205
- page_rank: -0.5111371436645006


#### Variables no tienen mayor correlación con status

Notamos que la mayoria de variables no tienen correlación con el status por lo cual lo vamos a descartar para nuestro modelo.

#### Variables con correlación entre variables
Todas estas correlaciones fueron tomadas del `final_report.html`

Entre ip y f26 se noto una correlación positiva significativa dentro de nuestro `final_report.html` notamos que era una corr > a .7

Entre nb_www y google_index se noto una correlación negativa relativamente fuerte el valor aproximado que se le asigna es de -0.3

Entre phish_hints y google_index se noto una correlación positiva relativamente fuerte el valor aproximado que se le asigna es de 0.35

Entre domain_age y page_rank se noto una correlación fuerte el valor aproximado que se le asigna es de 0.6

Debido a estas correlaciones se excluiran las variables ip y domain_age

#### Variables constantes

No encontramos variables constantes en `final_report.html`

In [94]:
filter_final_columns = filter(lambda x: x != "ip" and x != "domain_age" and x != "status", correlation_columns)
list(filter_final_columns)

['nb_www',
 'phish_hints',
 'nb_hyperlinks',
 'domain_in_title',
 'google_index',
 'page_rank',
 'f7',
 'f26']

In [109]:
# Create the dataset to feed the final profiles only with the correlation columns
final_dataset_model = final_dataset.drop(["status", "ip", "domain_age"], axis=1)


In [110]:
final_dataset_model.columns

Index(['nb_www', 'phish_hints', 'nb_hyperlinks', 'domain_in_title',
       'google_index', 'page_rank', 'f7', 'f26'],
      dtype='object')

# Modelo

## Separación train y test set

In [111]:
y = final_dataset["status"]
X = final_dataset_model

In [140]:
# X_train_beta, X_test, y_train_beta, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [114]:
# validation_rate = 15/70
# X_train, X_validation, y_train, y_validation = train_test_split(X_train_beta, y_train_beta, test_size=validation_rate, random_state=42)

In [141]:
def store_df(name, X, y):
    compression_opts = dict(method='zip',
                            archive_name=name+'_X.csv')  
    X.to_csv(name+'_X.zip', index=True,
              compression=compression_opts)  
    compression_opts = dict(method='zip',
                            archive_name=name+'_y.csv')  
    y.to_csv(name+'_y.zip', index=True,
              compression=compression_opts)  

In [142]:
store_df("test_values", X_test, y_test)
store_df("train_values", X_train, y_train)
# store_df("validate_values", X_validation, y_validation)

## Creación del modelo

In [143]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [144]:
# Entrenamos el modelo
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [145]:
# Realizamos predicciones
predictions = rf.predict(X_test)

In [149]:
y_test.shape

(3429,)

In [147]:
predictions.shape

(3429,)

In [150]:
confusion_matrix = confusion_matrix(y_test, predictions)
recall_score = recall_score(y_test, predictions, average=None)
precision_score = precision_score(y_test, predictions, average=None)
f1_score = f1_score(y_test, predictions, average=None)

In [151]:
print("Matrix de confusión")
confusion_matrix

Matrix de confusión


array([[1629,  103],
       [  86, 1611]])

In [152]:
print("recall_score")
recall_score

recall_score


array([0.94053118, 0.94932233])

In [153]:
print("precision_score")
precision_score

precision_score


array([0.94985423, 0.93990665])

In [154]:
print("f1_score")
f1_score

f1_score


array([0.94516971, 0.94459103])

# PREGUNTAS

1. `¿Cuál es el impacto de clasificar un sitio legítimo como Pishing?`
* El impacto es mínimo, puesto que lo peor que pasa es que el usuario no puede acceder con facilidad al sitio.

2. `¿Cuál es el impacto de clasificar un sitio de Pishing como legítimo?`
* Esto puede llegar a tener un impacto importante, puesto que puede causar que el personal de una organización ingrese a un sitio con phishing y vulnerar la seguridad de toda la organización.

3. `En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares de clasificación de pishing?`
* Consideramos que el recall es la mejor métrica para este problema, puesto que nos indica la habilidad del modelo de encontrar los positivos.

4. `¿Es necesaria la intervención de una persona humana en la decisión final de clasificación?`
* Sería imposible, debido a la corta vida de las páginas de phishing y a la enorme cantidad de variables que se deben de tomar en consideración para tomar una decisión acertada. En lo único en lo que podría intervenir una persona es en detectar cuando el modelo idenficó mal a un sitio y habilitarlo manualmente.